# Lineare Regression

Der Daten für den Verbrauchspreisindex sind für das Jahr 2019 nicht verfügbar. Daher werden zwei Optionen getestet: 

- Trainieren eines Modells mit den Jahren 2020–2023 (mit Preisindex).
- Trainieren eines Modells mit 2019–2023 (ohne Preisindex).

In [1]:
import pandas as pd

df = pd.read_csv("daten/Merged_BSR_24Uhr.csv")

In [2]:
df.describe()

,Schicht,Tour,Tonnage,Temperature_Max (°C),Rain_Sum (mm),Snowfall_Sum (cm),Wind_Speed_Max (km/h),Daylight_Duration (s),Temperature_Max (°C) 3-Day Avg,Rain_Sum (mm) 3-Day Avg,...,Feiertag_Pfingstmontag_shift_1,Feiertag_Pfingstmontag_shift_2,Feiertag_Tag der Arbeit_shift_1,Feiertag_Tag der Arbeit_shift_2,Feiertag_Tag der Befreiung_shift_1,Feiertag_Tag der Befreiung_shift_2,Feiertag_Tag der Deutschen Einheit_shift_1,Feiertag_Tag der Deutschen Einheit_shift_2,Feiertag_Zweiter Weihnachtstag_shift_1,Feiertag_Zweiter Weihnachtstag_shift_2
count,381661.000000,381661.000000,381661.000000,381661.000000,381661.000000,381661.000000,381661.000000,381661.000000,381661.000000,381661.000000,...,381661.0,381661.0,381661.0,381661.0,381661.0,381661.0,381661.0,381661.0,381661.0,381661.0
mean,1.018522,143.118574,12.125463,15.146716,1.643809,0.061519,19.949507,44466.500889,15.186362,1.519789,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.134828,529.435676,5.993363,8.440446,3.558954,0.305219,6.834062,11275.498180,8.316128,2.168854,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,1.000000,1.000000,0.100000,-7.200000,0.000000,0.000000,7.900000,27516.100000,-6.633333,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,1.000000,11.000000,8.080000,8.200000,0.000000,0.000000,15.100000,33653.080000,8.233333,0.066667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,1.000000,23.000000,11.100000,15.300000,0.200000,0.000000,18.800000,44861.230000,14.933333,0.633333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,1.000000,41.000000,16.910000,21.600000,1.600000,0.000000,23.700000,55288.220000,22.000000,2.166667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,2.000000,5513.000000,59.580000,37.700000,41.300000,4.900000,49.700000,60622.050000,33.900000,23.466667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381661 entries, 0 to 381660
Data columns (total 91 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Schicht                                     381661 non-null  int64  
 1   Tour                                        381661 non-null  int64  
 2   Tonnage                                     381661 non-null  float64
 3   Temperature_Max (°C)                        381661 non-null  float64
 4   Rain_Sum (mm)                               381661 non-null  float64
 5   Snowfall_Sum (cm)                           381661 non-null  float64
 6   Wind_Speed_Max (km/h)                       381661 non-null  float64
 7   Daylight_Duration (s)                       381661 non-null  float64
 8   Temperature_Max (°C) 3-Day Avg              381661 non-null  float64
 9   Rain_Sum (mm) 3-Day Avg                     381661 non-null  float64
 

### Entfernen des Verbauchspreisindex für das Jahr 209

In [4]:
print(df.columns.tolist())  # Gibt die exakten Spaltennamen aus


['Schicht', 'Tour', 'Tonnage', 'Temperature_Max (°C)', 'Rain_Sum (mm)', 'Snowfall_Sum (cm)', 'Wind_Speed_Max (km/h)', 'Daylight_Duration (s)', 'Temperature_Max (°C) 3-Day Avg', 'Rain_Sum (mm) 3-Day Avg', 'Snowfall_Sum (cm) 3-Day Avg', 'Wind_Speed_Max (km/h) 3-Day Avg', 'Daylight_Duration (s) 3-Day Avg', 'SPD', 'Grünen', 'CDU', 'Linke', 'AfD', 'FDP', 'Sonstige', 'Feiertag', 'Feiertag_Christi Himmelfahrt', 'Feiertag_Erster Weihnachtstag', 'Feiertag_Internationaler Frauentag', 'Feiertag_Karfreitag', 'Feiertag_Neujahr', 'Feiertag_Ostermontag', 'Feiertag_Pfingstmontag', 'Feiertag_Tag der Arbeit', 'Feiertag_Tag der Befreiung', 'Feiertag_Tag der Deutschen Einheit', 'Feiertag_Zweiter Weihnachtstag', 'Wochentag', '   Brot', 'Fleisch', '    Fisch', '    Molkerei-Eier', '    Speisefette-öle', '    Obst', '    Gemüse', '    Zucker, Marmelade, Honig', '    Fertigerichte', '  Kaffee-Tee', '    Wasser-Saft', 'TageZuvorNichtGeliefert', 'Ferientag', 'Saison_Frühling', 'Saison_Herbst', 'Saison_Sommer', 

In [5]:
df.columns = df.columns.str.strip()  # Entfernt alle führenden und nachfolgenden Leerzeichen

In [6]:
preisindex_spalten = ["Brot", "Fleisch", "Fisch", "Molkerei-Eier", 
                      "Speisefette-öle", "Obst", "Gemüse", "Zucker, Marmelade, Honig", 
                      "Fertigerichte", "Kaffee-Tee", "Wasser-Saft"]

df_cleaned = df.drop(columns=preisindex_spalten)

In [7]:
df_cleaned.head(30)  # Überprüfe, ob die Spalten entfernt wurden

,Schicht,Tour,Tonnage,Temperature_Max (°C),Rain_Sum (mm),Snowfall_Sum (cm),Wind_Speed_Max (km/h),Daylight_Duration (s),Temperature_Max (°C) 3-Day Avg,Rain_Sum (mm) 3-Day Avg,...,Feiertag_Pfingstmontag_shift_1,Feiertag_Pfingstmontag_shift_2,Feiertag_Tag der Arbeit_shift_1,Feiertag_Tag der Arbeit_shift_2,Feiertag_Tag der Befreiung_shift_1,Feiertag_Tag der Befreiung_shift_2,Feiertag_Tag der Deutschen Einheit_shift_1,Feiertag_Tag der Deutschen Einheit_shift_2,Feiertag_Zweiter Weihnachtstag_shift_1,Feiertag_Zweiter Weihnachtstag_shift_2
0,1,1,5.59,3.6,0.0,0.0,31.2,27967.86,6.133333,0.633333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,4,3.23,3.6,0.0,0.0,31.2,27967.86,6.133333,0.633333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,5,5.68,3.6,0.0,0.0,31.2,27967.86,6.133333,0.633333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,6,5.48,3.6,0.0,0.0,31.2,27967.86,6.133333,0.633333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,7,7.84,3.6,0.0,0.0,31.2,27967.86,6.133333,0.633333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1,9,4.63,3.6,0.0,0.0,31.2,27967.86,6.133333,0.633333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1,10,4.38,3.6,0.0,0.0,31.2,27967.86,6.133333,0.633333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1,12,9.50,3.6,0.0,0.0,31.2,27967.86,6.133333,0.633333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1,1,24.00,3.6,0.0,0.0,31.2,27967.86,6.133333,0.633333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1,2,20.70,3.6,0.0,0.0,31.2,27967.86,6.133333,0.633333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
df_cleaned.columns

Index(['Schicht', 'Tour', 'Tonnage', 'Temperature_Max (°C)', 'Rain_Sum (mm)',
       'Snowfall_Sum (cm)', 'Wind_Speed_Max (km/h)', 'Daylight_Duration (s)',
       'Temperature_Max (°C) 3-Day Avg', 'Rain_Sum (mm) 3-Day Avg',
       'Snowfall_Sum (cm) 3-Day Avg', 'Wind_Speed_Max (km/h) 3-Day Avg',
       'Daylight_Duration (s) 3-Day Avg', 'SPD', 'Grünen', 'CDU', 'Linke',
       'AfD', 'FDP', 'Sonstige', 'Feiertag', 'Feiertag_Christi Himmelfahrt',
       'Feiertag_Erster Weihnachtstag', 'Feiertag_Internationaler Frauentag',
       'Feiertag_Karfreitag', 'Feiertag_Neujahr', 'Feiertag_Ostermontag',
       'Feiertag_Pfingstmontag', 'Feiertag_Tag der Arbeit',
       'Feiertag_Tag der Befreiung', 'Feiertag_Tag der Deutschen Einheit',
       'Feiertag_Zweiter Weihnachtstag', 'Wochentag',
       'TageZuvorNichtGeliefert', 'Ferientag', 'Saison_Frühling',
       'Saison_Herbst', 'Saison_Sommer', 'Saison_Winter', 'year', 'month',
       'day', 'Tonnage_dealy_1', 'Tonnage_dealy_2', 'Tonnage_dealy_3'

## Daten bereinigen & Feature Engineering

In [9]:
# Fehlende Werte prüfen
df_cleaned.isnull().sum()

Schicht                                       0
Tour                                          0
Tonnage                                       0
Temperature_Max (°C)                          0
Rain_Sum (mm)                                 0
                                             ..
Feiertag_Tag der Befreiung_shift_2            0
Feiertag_Tag der Deutschen Einheit_shift_1    0
Feiertag_Tag der Deutschen Einheit_shift_2    0
Feiertag_Zweiter Weihnachtstag_shift_1        0
Feiertag_Zweiter Weihnachtstag_shift_2        0
Length: 80, dtype: int64

In [10]:
df_cleaned.isna().sum()

Schicht                                       0
Tour                                          0
Tonnage                                       0
Temperature_Max (°C)                          0
Rain_Sum (mm)                                 0
                                             ..
Feiertag_Tag der Befreiung_shift_2            0
Feiertag_Tag der Deutschen Einheit_shift_1    0
Feiertag_Tag der Deutschen Einheit_shift_2    0
Feiertag_Zweiter Weihnachtstag_shift_1        0
Feiertag_Zweiter Weihnachtstag_shift_2        0
Length: 80, dtype: int64

In [11]:
df_cleaned.head()

,Schicht,Tour,Tonnage,Temperature_Max (°C),Rain_Sum (mm),Snowfall_Sum (cm),Wind_Speed_Max (km/h),Daylight_Duration (s),Temperature_Max (°C) 3-Day Avg,Rain_Sum (mm) 3-Day Avg,...,Feiertag_Pfingstmontag_shift_1,Feiertag_Pfingstmontag_shift_2,Feiertag_Tag der Arbeit_shift_1,Feiertag_Tag der Arbeit_shift_2,Feiertag_Tag der Befreiung_shift_1,Feiertag_Tag der Befreiung_shift_2,Feiertag_Tag der Deutschen Einheit_shift_1,Feiertag_Tag der Deutschen Einheit_shift_2,Feiertag_Zweiter Weihnachtstag_shift_1,Feiertag_Zweiter Weihnachtstag_shift_2
0,1,1,5.59,3.6,0.0,0.0,31.2,27967.86,6.133333,0.633333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,4,3.23,3.6,0.0,0.0,31.2,27967.86,6.133333,0.633333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,5,5.68,3.6,0.0,0.0,31.2,27967.86,6.133333,0.633333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,6,5.48,3.6,0.0,0.0,31.2,27967.86,6.133333,0.633333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,7,7.84,3.6,0.0,0.0,31.2,27967.86,6.133333,0.633333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Daten aufteilen

In [12]:
# Trainingsdaten (2019, 2020, 2021)
train_data = df[df['year'].isin([2019, 2020, 2021])]

# Testdaten (2022)
test_data = df[df['year'] == 2022]

# Validierungsdaten (2023)
validation_data = df[df['year'] == 2023]

In [13]:
# Zielvariable
y_train = train_data["Tonnage"]
y_test = test_data["Tonnage"]
y_validation = validation_data["Tonnage"]

# Features
X_train = train_data.drop(columns=['Tonnage', 'year', 'day'])
X_test = test_data.drop(columns=['Tonnage', 'year', 'day'])
X_validation = validation_data.drop(columns=['Tonnage', 'year', 'day'])

## Standardisierung

In [14]:
from sklearn.preprocessing import StandardScaler

# Zunächst den StandardScaler erstellen
scaler = StandardScaler()

# Nur numerische Features, die standardisiert werden sollen
numerical_features = [
    'Rain_Sum (mm)', 
    'Snowfall_Sum (cm)', 
    'Wind_Speed_Max (km/h)', 
    'Daylight_Duration (s)',
    'Temperature_Max (°C) 3-Day Avg', 
    'Rain_Sum (mm) 3-Day Avg',
    'Snowfall_Sum (cm) 3-Day Avg', 
    'Wind_Speed_Max (km/h) 3-Day Avg',
    'Daylight_Duration (s) 3-Day Avg'
]

# Standardisierung der Trainingsdaten
X_train[numerical_features] = scaler.fit_transform(X_train[numerical_features])

# Standardisierung der Testdaten (mit denselben Parametern wie für die Trainingsdaten)
X_test[numerical_features] = scaler.transform(X_test[numerical_features])

# Standardisierung der Validierungsdaten (mit denselben Parametern wie für die Trainingsdaten)
X_validation[numerical_features] = scaler.transform(X_validation[numerical_features])


In [15]:
# Schau dir die ersten 5 Zeilen des standardisierten Trainingsdatensatzes an
X_train.head()


,Schicht,Tour,Temperature_Max (°C),Rain_Sum (mm),Snowfall_Sum (cm),Wind_Speed_Max (km/h),Daylight_Duration (s),Temperature_Max (°C) 3-Day Avg,Rain_Sum (mm) 3-Day Avg,Snowfall_Sum (cm) 3-Day Avg,...,Feiertag_Pfingstmontag_shift_1,Feiertag_Pfingstmontag_shift_2,Feiertag_Tag der Arbeit_shift_1,Feiertag_Tag der Arbeit_shift_2,Feiertag_Tag der Befreiung_shift_1,Feiertag_Tag der Befreiung_shift_2,Feiertag_Tag der Deutschen Einheit_shift_1,Feiertag_Tag der Deutschen Einheit_shift_2,Feiertag_Zweiter Weihnachtstag_shift_1,Feiertag_Zweiter Weihnachtstag_shift_2
0,1,1,3.6,-0.466407,-0.200949,1.76435,-1.461315,-1.073739,-0.376103,-0.246033,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,4,3.6,-0.466407,-0.200949,1.76435,-1.461315,-1.073739,-0.376103,-0.246033,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,5,3.6,-0.466407,-0.200949,1.76435,-1.461315,-1.073739,-0.376103,-0.246033,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,6,3.6,-0.466407,-0.200949,1.76435,-1.461315,-1.073739,-0.376103,-0.246033,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,7,3.6,-0.466407,-0.200949,1.76435,-1.461315,-1.073739,-0.376103,-0.246033,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# Zähle die NaN-Werte in jeder Spalte des X_train-Datensatzes
na_counts = X_train.isna().sum()

# Zeige die Anzahl der NaN-Werte für jede Spalte an
print(na_counts)


Schicht                                       0
Tour                                          0
Temperature_Max (°C)                          0
Rain_Sum (mm)                                 0
Snowfall_Sum (cm)                             0
                                             ..
Feiertag_Tag der Befreiung_shift_2            0
Feiertag_Tag der Deutschen Einheit_shift_1    0
Feiertag_Tag der Deutschen Einheit_shift_2    0
Feiertag_Zweiter Weihnachtstag_shift_1        0
Feiertag_Zweiter Weihnachtstag_shift_2        0
Length: 88, dtype: int64


## Modell trainieren

In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Schritt 1: Lineares Regressionsmodell erstellen
model = LinearRegression()

In [20]:
X_train = X_train.dropna()
y_train = y_train[X_train.index]  # Anpassung der Zielvariable


In [21]:
# Schritt 2: Modell mit den Trainingsdaten trainieren
model.fit(X_train, y_train)


LinearRegression()

In [22]:
# Schritt 3: Vorhersagen auf den Testdaten und Validierungsdaten treffen
y_pred_test = model.predict(X_test)
y_pred_validation = model.predict(X_validation)

In [23]:
# Schritt 4: Modell evaluieren

# Auf den Testdaten
mse_test = mean_squared_error(y_test, y_pred_test)  # Mean Squared Error
r2_test = r2_score(y_test, y_pred_test)  # R² Score

# Auf den Validierungsdaten
mse_validation = mean_squared_error(y_validation, y_pred_validation)
r2_validation = r2_score(y_validation, y_pred_validation)

In [24]:
# Ergebnisse ausgeben
print(f"Testdaten - MSE: {mse_test}, R²: {r2_test}")
print(f"Validierungsdaten - MSE: {mse_validation}, R²: {r2_validation}")

Testdaten - MSE: 19.808467883838958, R²: 0.417109332720189
Validierungsdaten - MSE: 22.715064286593194, R²: 0.40398451390790435


## Standardisierung (Z-Transformation) 

- Zielvariable sollt enicht standardisiert bzw. normalisiert werden

Standardisierung (StandardScaler):

    Die Standardisierung transformiert die Daten so, dass sie eine Mittelwert von 0 und eine Standardabweichung von 1 haben. Es wird keine Skalierung auf einen festen Bereich angewendet.

    Wann ist es sinnvoll?
        Wenn die Daten eine normalverteilte Verteilung haben oder die Verteilung unbekannt ist.
        Für Modelle, die auf der Berechnung von Abständen oder Gradienten basieren, wie z. B. lineare Regression, logistische Regression, Support Vector Machines (SVM), k-nearest neighbors (k-NN) und neuronale Netze.

In [80]:
from sklearn.preprocessing import StandardScaler

# Definiere die numerischen Spalten, die standardisiert werden sollen
numerical_features = [
    'Rain_Sum (mm)', 
    'Snowfall_Sum (cm)', 
    'Wind_Speed_Max (km/h)', 
    'Daylight_Duration (s)',
    'Temperature_Max (°C) 3-Day Avg', 
    'Rain_Sum (mm) 3-Day Avg',
    'Snowfall_Sum (cm) 3-Day Avg', 
    'Wind_Speed_Max (km/h) 3-Day Avg',
    'Daylight_Duration (s) 3-Day Avg'
]

# Erstelle eine Kopie des DataFrames, um die Originaldaten nicht zu verändern
df_standardized = df.copy()

# Standardisiere die numerischen Spalten
scaler = StandardScaler()
df_standardized[numerical_features] = scaler.fit_transform(df_standardized[numerical_features])

# Jetzt sind die numerischen Spalten standardisiert
df_standardized.head()


,Schicht,Tour,Tonnage,Temperature_Max (°C),Rain_Sum (mm),Snowfall_Sum (cm),Wind_Speed_Max (km/h),Daylight_Duration (s),Temperature_Max (°C) 3-Day Avg,Rain_Sum (mm) 3-Day Avg,...,Feiertag_Pfingstmontag_shift_1,Feiertag_Pfingstmontag_shift_2,Feiertag_Tag der Arbeit_shift_1,Feiertag_Tag der Arbeit_shift_2,Feiertag_Tag der Befreiung_shift_1,Feiertag_Tag der Befreiung_shift_2,Feiertag_Tag der Deutschen Einheit_shift_1,Feiertag_Tag der Deutschen Einheit_shift_2,Feiertag_Zweiter Weihnachtstag_shift_1,Feiertag_Zweiter Weihnachtstag_shift_2
0,1,1,5.59,3.6,-0.46188,-0.201558,1.64624,-1.463231,-1.088612,-0.408721,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,4,3.23,3.6,-0.46188,-0.201558,1.64624,-1.463231,-1.088612,-0.408721,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,5,5.68,3.6,-0.46188,-0.201558,1.64624,-1.463231,-1.088612,-0.408721,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,6,5.48,3.6,-0.46188,-0.201558,1.64624,-1.463231,-1.088612,-0.408721,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,7,7.84,3.6,-0.46188,-0.201558,1.64624,-1.463231,-1.088612,-0.408721,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [81]:
df = df_standardized

In [82]:
# Zielvariable
y = df["Tonnage"]

In [83]:
# Features festlegen (alle Spalten außer Zielvariable und Datumsmerkmale)
features_to_drop = ['Tonnage', 'year', 'day']

# Features für das Modell
X = df.drop(columns=features_to_drop)


## Daten aufteilen nach Jahren

In [84]:
# Trainingsdaten (2019, 2020, 2021)
train_data = df[df['year'].isin([2019, 2020, 2021])]

# Testdaten (2022)
test_data = df[df['year'] == 2022]

# Validierungsdaten (2023)
validation_data = df[df['year'] == 2023]

## Features und Zielvariable für jedes Set trennen:

In [85]:
# Zielvariable
y_train = train_data["Tonnage"]
y_test = test_data["Tonnage"]
y_val = validation_data["Tonnage"]

# Features
X_train = train_data.drop(columns=['Tonnage', 'year', 'day'])
X_test = test_data.drop(columns=['Tonnage', 'year', 'day'])
X_val = validation_data.drop(columns=['Tonnage', 'year', 'day'])


In [87]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Modell erstellen
model = LinearRegression()

# Modell mit Trainingsdaten trainieren
model.fit(X_train, y_train)



ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
# Vorhersagen auf Test- und Validierungsdaten machen
y_pred_test = model.predict(X_test)
y_pred_validation = model.predict(X_validation)

# Modellbewertung auf Testdaten
mse_test = mean_squared_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)

# Modellbewertung auf Validierungsdaten
mse_validation = mean_squared_error(y_validation, y_pred_validation)
r2_validation = r2_score(y_validation, y_pred_validation)

print("Testdaten - MSE:", mse_test, "R^2:", r2_test)
print("Validierungsdaten - MSE:", mse_validation, "R^2:", r2_validation)


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Wähle die Features und das Ziel (z.B. Tonnage)
X = df_cleaned.drop(columns=['Tonnage', 'year', 'month', 'day'])  # Entferne die Zielspalte und Datumsmerkmale
y = df_cleaned['Tonnage']  # Zielvariable (Tonnage)

# Standardisieren (Z-Transformation)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-Test-Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


# Training

## Trainings- & Testdaten aufteilen

In [ ]:
from sklearn.model_selection import train_test_split

# Zielvariable
y = df["Tonnage"]

# Unabhängige Variablen (alle außer "Tonnage")
X = df.drop(columns=["Tonnage"])

# Trainings- und Testdaten splitten (80% Training, 20% Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Trainingsdaten: {X_train.shape}, Testdaten: {X_test.shape}")


##  Lineares Regressionsmodell trainieren

In [ ]:
from sklearn.linear_model import LinearRegression

# Lineares Modell
model = LinearRegression()
model.fit(X_train, y_train)

# Vorhersagen
y_pred = model.predict(X_test)

# Modellbewertung
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2: {r2}")


## Modell bewerten

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Modellbewertung
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"📊 Modellbewertung:")
print(f"➡️ Mean Absolute Error (MAE): {mae:.2f}")
print(f"➡️ Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"➡️ R² Score: {r2:.4f}")
